In [ ]:
    # !wget http://www.gutenberg.org/cache/epub/16457/pg16457.txt

In [ ]:
corpus = []

In [ ]:
from collections import defaultdict

In [ ]:
with open('pg16457.txt', 'r') as f:
    text = ' '.join(f.readlines())

In [ ]:
corpus = text.split(' ')

In [ ]:
len(corpus)

In [ ]:
def refactor(text):
    not_allowed = """\\-"_.•-—‘’',“”~@#[]%{}?°1234567890;!^/&*()_=+\n"""
    return ''.join(str(c) for c in text if not not_allowed.__contains__(c))

In [ ]:
corpus = [refactor(text) for text in corpus]

In [ ]:
corpus = [text.lower() for text in corpus if len(text) > 0]

In [ ]:
corpus_dict = dict()

In [ ]:
for text in corpus:
    if corpus_dict.__contains__(text):
        corpus_dict[text] += 1
    else:
        corpus_dict[text] = 1

In [ ]:
len(corpus_dict)

In [ ]:
corpus.sort()

In [ ]:
len(corpus)

In [ ]:
word_freqs = defaultdict(int)

In [ ]:
for text in corpus:
    word_freqs[text] += 1

In [ ]:
print(word_freqs)

In [ ]:
print(corpus_dict)

In [ ]:
print(len(word_freqs.keys()))

In [ ]:
print(word_freqs['unsolicited'])

In [ ]:
cnt = 0
for text in word_freqs.keys():
    if word_freqs[text] != corpus_dict[text]:
        print(text, 'wf', word_freqs[text], 'cd', corpus_dict[text])
    else:
        cnt += 1

In [ ]:
print(cnt)

In [ ]:
alphabet = []

In [ ]:
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()

print(alphabet)

In [ ]:
len(alphabet)


In [ ]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

In [ ]:
print(vocab)

In [ ]:
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

In [ ]:
print(splits)

In [ ]:
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [ ]:
pair_scores = compute_pair_scores(splits)

for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

In [ ]:
def get_best_pair(pair_scores):
    best_pair = ""
    best_score = None

    for pair, score in pair_scores.items():
        if best_score is None or best_pair < score:
            best_score = pair
            best_pair = score
    return best_pair, best_score


In [ ]:
print(splits)

In [ ]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [ ]:
print(vocab)

In [ ]:
print(compute_pair_scores(splits))

In [ ]:
get_best_pair(compute_pair_scores(splits))

In [ ]:
splits = merge_pair("a", "##b", splits)
splits["about"]

In [ ]:
vocab_size = 1000

In [ ]:
print(len(vocab))

In [ ]:
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    print(best_pair[0] + best_pair[1][2:])
    vocab.append(new_token)

In [ ]:
print(splits)

In [ ]:
vocab

In [ ]:
vocab.sort()

In [ ]:
print(vocab)

In [ ]:
splits

In [ ]:
print(splits)

In [ ]:
print(vocab)

In [ ]:
%%sql


In [ ]:
vocab_set = set(vocab)

In [133]:
def tokenize(sentence):
    n = len(sentence)
    results = []
    
    result_list = [None  for _ in range(n+1) ]
    result_list[0] = []
    
    has = [[False for _1 in range(n+1)] for _2 in range(n+1)]
    for i in range(0,n+1):
        for j in range(i+1,n+1):
            if i > 0:
                if vocab_set.__contains__("##" + sentence[i:j]) :
                    # print('has', "##" + sentence[i:j])
                    has[i][j] = True
            else :
                if vocab_set.__contains__(sentence[i:j]) :
                    # print('has', sentence[i:j])
                    has[i][j] = True
            # if sentence[i:j] == False :
                # print('NOT', sentence[i:j])
        print(has[i])        
    
    for i in range(1, n+1) :
        for j in range(0, i) :
            if result_list[j] is not None : 
                if  has[i][j] :
                    for res in result_list[j]:
                        res.append(sentence[j:i])
                        if result_list[i] is not None :
                            result_list[i].extend(res)
                        else :
                            result_list[i] = [res]
    return result_list


In [134]:
tokenize('hello')

[False, True, False, False, False, False]
[False, False, True, False, False, False]
[False, False, False, True, False, False]
[False, False, False, False, True, False]
[False, False, False, False, False, True]
[False, False, False, False, False, False]


[[], None, None, None, None, None]

In [112]:
vocab_set.__contains__('hel')

False

In [ ]:
# def tokenize(text):
#     pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
#     pre_tokenized_text = [word for word, offset in pre_tokenize_result]
#     splits = [[l for l in word] for word in pre_tokenized_text]
#     for pair, merge in merges.items():
#         for idx, split in enumerate(splits):
#             i = 0
#             while i < len(split) - 1:
#                 if split[i] == pair[0] and split[i + 1] == pair[1]:
#                     split = split[:i] + [merge] + split[i + 2 :]
#                 else:
#                     i += 1
#             splits[idx] = split
# 
#     return sum(splits, [])